In [2357]:
import numpy as np
import matplotlib.pyplot as plt

In [2358]:
from loaders import clean_text_for_pattern_letter
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
500
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [2359]:
def predict_train(input_str, letters, max_index:int = 4,):
    next_index: dict[str, int] = {}
    index_by_prev: dict[str, dict] = {}

    StatsDict: dict[str, dict] = {}

    for letter in letters:
        next_index[letter] = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]
        
        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict[stats_key].setdefault(next_inp, 0)

        StatsDict[stats_key][next_inp] += 1

        by_prev_key = prev_inp + str(prev_index) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1

        by_prev_key = prev_inp + str(0) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1
        
        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index:
            next_index[prev_inp] = 1
    
    return StatsDict, index_by_prev

In [2360]:
def predict(input_str, TF:list, StatsDict:dict, index_by_prev:dict, max_index, err = 0,):
    prev_index_l = [x for x in range(1, max_index+1)]

    output = []

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            # B B {'B': 1} [3] {3: 1} [3] False
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [2361]:
# train
max_index_L1 = 8

index_by_prev_L1: dict[str, dict] = {}
StatsDict_L1: dict[str, dict] = {}

StatsDict, index_by_prev_L1 = predict_train(letters=letters, input_str=input_str, max_index=max_index_L1)

In [2362]:
for sdk, sdv in index_by_prev_L1.items():
    print(sdk, sdv)
    print('')

t1:h {1: 1}

t0:h {1: 2, 2: 2, 3: 2, 4: 1, 5: 2, 7: 1, 8: 1}

h1:e {1: 1}

h0:e {1: 1, 3: 2, 4: 4, 7: 1, 2: 2, 6: 2, 8: 1}

e1:  {1: 1, 5: 1}

e0:  {1: 3, 3: 1, 4: 3, 7: 3, 2: 3, 5: 3, 8: 1}

 1:s {1: 1}

 0:s {1: 1, 7: 1, 8: 1, 2: 2, 3: 1}

s1:k {1: 1}

s0:k {1: 1}

k1:y {1: 1}

k0:y {1: 1}

y1:  {2: 1}

y0:  {2: 1, 5: 1, 1: 1, 7: 1}

 2:a {1: 1}

 0:a {1: 3, 3: 4, 7: 1, 2: 1, 6: 1, 4: 2}

a1:b {1: 1}

a0:b {1: 1}

b1:o {1: 1}

b0:o {1: 1, 2: 1, 6: 1}

o1:v {1: 1}

o0:v {1: 1}

v1:e {2: 1}

v0:e {2: 1, 7: 2, 1: 1}

e2:  {3: 1, 2: 1, 7: 1}

 3:t {2: 1, 7: 1, 8: 1, 6: 1}

 0:t {2: 3, 4: 2, 5: 2, 6: 2, 7: 2, 3: 1, 8: 1, 1: 1}

t2:h {2: 1, 1: 1, 8: 1, 5: 1}

h2:e {3: 1, 2: 1}

e3:  {4: 2, 5: 1, 2: 1}

 4:p {1: 1}

 0:p {1: 1, 5: 1, 8: 1}

p1:o {2: 1}

p0:o {2: 1}

o2:r {1: 1}

o0:r {1: 1, 2: 2, 8: 1, 5: 2}

r1:t {3: 1}

r0:t {3: 1}

t3:  {5: 1}

t0:  {5: 1, 1: 1, 8: 1}

 5:w {1: 1}

 0:w {1: 1, 2: 2, 4: 1, 7: 1, 8: 1, 3: 1}

w1:a {2: 1}

w0:a {2: 2, 6: 1, 5: 1, 1: 1}

a2:s {2: 1}

a0:s {2

In [2363]:
for sdk, sdv in StatsDict.items():
    print(sdk, sdv)
    print('')

t1:h1 {'e': 1}

h1:e1 {' ': 1}

e1: 1 {'s': 1}

 1:s1 {'k': 1}

s1:k1 {'y': 1}

k1:y1 {' ': 1}

y1: 2 {'a': 1}

 2:a1 {'b': 1}

a1:b1 {'o': 1}

b1:o1 {'v': 1}

o1:v1 {'e': 1}

v1:e2 {' ': 1}

e2: 3 {'t': 1}

 3:t2 {'h': 1}

t2:h2 {'e': 1}

h2:e3 {' ': 1}

e3: 4 {'p': 1, 'f': 1}

 4:p1 {'o': 1}

p1:o2 {'r': 1}

o2:r1 {'t': 1}

r1:t3 {' ': 1}

t3: 5 {'w': 1}

 5:w1 {'a': 1}

w1:a2 {'s': 1}

a2:s2 {' ': 1}

s2: 6 {'t': 1}

 6:t4 {'h': 1}

t4:h3 {'e': 1}

h3:e4 {' ': 1}

e4: 7 {'c': 1}

 7:c1 {'o': 1}

c1:o3 {'l': 1}

o3:l1 {'o': 1}

l1:o4 {'r': 1}

o4:r2 {' ': 2}

r2: 8 {'o': 1}

 8:o5 {'f': 1}

o5:f1 {' ': 1}

f1: 1 {'t': 1}

 1:t5 {'e': 1}

t5:e5 {'l': 1}

e5:l2 {'e': 1}

l2:e6 {'v': 1}

e6:v2 {'i': 1}

v2:i1 {'s': 1}

i1:s3 {'i': 1}

s3:i2 {'o': 1}

i2:o6 {'n': 1}

o6:n1 {',': 1}

n1:,1 {' ': 1}

,1: 2 {'t': 1}

 2:t6 {'u': 1}

t6:u1 {'n': 1}

u1:n2 {'e': 1}

n2:e7 {'d': 1}

e7:d1 {' ': 1}

d1: 3 {'t': 1}

 3:t7 {'o': 1}

t7:o7 {' ': 1}

o7: 4 {'a': 1}

 4:a3 {' ': 1, 'r': 1}

a3: 5 {'

In [2364]:
# predict
TF = []

TF, err, out_L1 = predict(input_str=input_str, TF=TF, StatsDict=StatsDict, index_by_prev=index_by_prev_L1, max_index=max_index_L1)

print(err, err / (len(input_str) / 100) )

e e {'e': 6, 'i': 1} [1, 2, 8, 5, 3, 4, 7] {1: 2, 2: 2, 8: 1, 5: 2, 3: 2, 4: 1, 7: 1} [1, 2, 3, 4, 5, 6, 7, 8] False
    {' ': 4, 'r': 1, 'a': 2, 't': 1} [1, 3, 2, 6, 4, 8] {1: 1, 3: 2, 2: 2, 6: 1, 4: 4, 8: 1} [1, 2, 8, 5, 3, 4, 7] False
d s {'s': 1, 'd': 2, 'p': 1, 'f': 1, 't': 1, 'c': 1} [1, 5, 4, 2, 3, 7] {1: 3, 5: 3, 4: 3, 2: 2, 3: 1, 7: 3} [1, 3, 2, 6, 4, 8] False
k k {'k': 1, 'h': 1, 'p': 1, 'a': 1} [1, 2, 8] {1: 1, 2: 2, 8: 1} [1, 5, 4, 2, 3, 7] False
y y {'y': 1} [1] {1: 1} [1, 2, 8] False
    {' ': 1} [1] {1: 1} [1] False
a a {'a': 1} [2] {2: 1} [1] False
b b {'b': 1} [1] {1: 1} [2] False
o o {'o': 1} [1] {1: 1} [1] False
v v {'v': 1} [1] {1: 1} [1] False
e e {'e': 1} [1] {1: 1} [1] False
    {' ': 1} [2] {2: 1} [1] False
t t {'t': 1, 'i': 1, 'c': 1} [3, 2, 7] {3: 1, 2: 1, 7: 1} [2] False
h h {'h': 3, 'o': 1} [2, 7, 8, 6] {2: 1, 7: 2, 8: 1, 6: 2} [3, 2, 7] False
e e {'e': 3, 'r': 1, 'i': 1} [2, 1, 8, 5, 7] {2: 2, 1: 1, 8: 1, 5: 1, 7: 1} [2, 7, 8, 6] False
    {' ': 4, 'r': 1, 

In [2365]:
def transition_freedom(input_str: str):
    transition_freedom: dict[str, dict] = {}

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        transition_freedom.setdefault(prev_inp, {})
        transition_freedom[prev_inp].setdefault(inp, 0)

    # print(transition_freedom)

    for tf_key, tf in transition_freedom.items():
        transition_freedom[tf_key] = len(tf)
    
    # print(transition_freedom)

    return transition_freedom
        
TF_per_symbol = transition_freedom(input_str)
TF = []
for inp in input_str:
    TF.append(TF_per_symbol[inp])
print(len(TF), TF)

500 [12, 6, 15, 22, 12, 4, 5, 22, 11, 2, 16, 3, 15, 22, 12, 6, 15, 22, 4, 16, 11, 12, 22, 5, 11, 12, 22, 12, 6, 15, 22, 8, 16, 8, 16, 11, 22, 16, 4, 22, 12, 15, 8, 15, 3, 11, 12, 11, 16, 10, 2, 22, 12, 6, 10, 15, 7, 22, 12, 16, 22, 11, 22, 7, 15, 11, 7, 22, 8, 6, 11, 10, 10, 15, 8, 2, 22, 2, 11, 12, 2, 12, 22, 10, 16, 12, 22, 8, 11, 4, 15, 22, 11, 2, 5, 22, 6, 12, 11, 10, 3, 2, 2, 22, 8, 11, 12, 15, 22, 6, 15, 11, 11, 7, 22, 12, 16, 5, 15, 16, 10, 15, 22, 12, 11, 5, 2, 22, 11, 12, 22, 6, 15, 22, 12, 6, 16, 6, 8, 7, 15, 11, 15, 7, 22, 6, 11, 12, 22, 5, 11, 5, 22, 12, 6, 11, 16, 6, 3, 6, 22, 12, 6, 15, 22, 8, 11, 16, 5, 7, 22, 11, 11, 16, 6, 10, 7, 22, 12, 6, 15, 22, 7, 16, 16, 11, 22, 16, 4, 22, 12, 6, 15, 22, 8, 6, 11, 12, 2, 22, 2, 11, 12, 2, 12, 22, 8, 11, 4, 15, 22, 5, 5, 22, 2, 16, 7, 5, 2, 12, 22, 7, 15, 3, 15, 8, 16, 4, 15, 7, 22, 12, 6, 11, 12, 22, 5, 11, 12, 12, 11, 3, 15, 22, 7, 11, 6, 3, 22, 7, 15, 4, 11, 8, 11, 15, 10, 8, 5, 2, 2, 22, 11, 12, 22, 5, 11, 12, 22, 11, 22, 12, 4

In [2366]:
TF = np.array(TF, dtype=float)
# print(TF)
# TF = np.pad(TF, (1, 1), constant_values=[1,1])
# # print(TF)
# TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (1, 0))[:-1] 
# # TF = np.pad(TF, (1, 0))[:-1] - TF
# # TF = TF - np.pad(TF, (0, 1))[-1:]
# # print(TF)
# TF[np.where(TF < 0)] = 0

for idx in range(1):
    TF = TF - np.pad(TF, (1, 0))[:-1]
    TF[np.where(TF < 0)] = 0
    TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF = np.abs(TF)
# TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF[np.where(TF <= 0.5)] = 0
# print(TF.size, TF.max(), TF.min())
# TF[np.where(TF <= np.mean(TF))] = 0

# fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
# # axs.plot(np.arange(TF.size), TF)
# markerline, stemline, baseline, = axs.stem(np.arange(TF.size), TF)
# plt.setp(markerline, markersize = 1)
# axs.xaxis.set_ticks(np.arange(len(input_str)) ,input_str)
# axs.xaxis.grid(True)

plt.show()

In [2367]:
upstream_inp: dict[int, list] = {}

temp_seq = []

for inp_idx, inp in enumerate(input_str):
    tf = TF[inp_idx]
    
    if tf > 0:
        upstream_inp.setdefault(inp_idx, temp_seq.copy())
        temp_seq.clear()

    temp_seq.append(inp)

for item, val in upstream_inp.items():
    print(item, val)
    
print(len(upstream_inp))

0 []
2 ['t', 'h']
7 ['e', ' ', 's', 'k', 'y']
10 [' ', 'a', 'b']
12 ['o', 'v']
16 ['e', ' ', 't', 'h']
19 ['e', ' ', 'p']
22 ['o', 'r', 't']
26 [' ', 'w', 'a', 's']
29 [' ', 't', 'h']
36 ['e', ' ', 'c', 'o', 'l', 'o', 'r']
39 [' ', 'o', 'f']
51 [' ', 't', 'e', 'l', 'e', 'v', 'i', 's', 'i', 'o', 'n', ',']
57 [' ', 't', 'u', 'n', 'e', 'd']
62 [' ', 't', 'o', ' ', 'a']
67 [' ', 'd', 'e', 'a', 'd']
76 [' ', 'c', 'h', 'a', 'n', 'n', 'e', 'l', '.']
78 [' ', '"']
81 ['i', 't', "'"]
82 ['s']
86 [' ', 'n', 'o', 't']
90 [' ', 'l', 'i', 'k']
95 ['e', ' ', 'i', "'", 'm']
103 [' ', 'u', 's', 'i', 'n', 'g', ',', '"']
110 [' ', 'c', 'a', 's', 'e', ' ', 'h']
114 ['e', 'a', 'r', 'd']
118 [' ', 's', 'o', 'm']
127 ['e', 'o', 'n', 'e', ' ', 's', 'a', 'y', ',']
130 [' ', 'a', 's']
132 [' ', 'h']
136 ['e', ' ', 's', 'h']
144 ['o', 'u', 'l', 'd', 'e', 'r', 'e', 'd']
148 [' ', 'h', 'i', 's']
152 [' ', 'w', 'a', 'y']
160 [' ', 't', 'h', 'r', 'o', 'u', 'g', 'h']
163 [' ', 't', 'h']
170 ['e', ' ', 'c', 'r', 'o',

In [2368]:
letters_dict: dict[str, int] = {}
for let_idx, letter in enumerate(letters):
    letters_dict.setdefault(letter, let_idx)

upstream_patterns = []

for upstream_letters in upstream_inp.values():
    upstream_pattern = np.zeros(len(letters))

    for letter in upstream_letters:
        pattern_idx = letters_dict.get(letter, 0)

        upstream_pattern[pattern_idx] = 1
    
    upstream_patterns.append(upstream_pattern)

upstream_patterns = np.array(upstream_patterns)

for up_ptr in upstream_patterns:
    print(up_ptr.astype(int).tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0,

In [2369]:
layer_size = 32

I = np.zeros(upstream_patterns.shape[1])
O = np.zeros(layer_size)
W = np.ones((O.size, I.size))
Trace = np.zeros((O.size, I.size))
print('W', W.shape, W.max(), W.min(), np.median(W))

k = 1
push_delta = 0.4

prec = 1e-30

epoch_cnt = 100
eps0 = 0.02

eps = 0.05
rate_drop = 0.0

AVG_O = np.zeros(O.shape)
AVG_O_CNT = np.zeros(O.shape)

samples_idx = np.arange(upstream_patterns.shape[0])

for epoch_idx in range(epoch_cnt):
    np.random.shuffle(samples_idx)

    for i_idx in samples_idx:
        I = upstream_patterns[i_idx]

        # infer
        O = np.dot(W, I)

        # train
        winner_ids = np.argsort(O)[::-1][:1+k]
        winner = winner_ids[0]
        loosers = winner_ids[1:1+k]

        # update
        W[winner] = (1 - eps) * W[winner] + eps * (I - 0.5) + Trace[winner]*eps
        W[winner] = np.clip(W[winner], -1, 1)

        # W[loosers] = (1 - eps) * W[loosers] + (eps * (I - 0.5) * -push_delta) + Trace[loosers]*eps
        # W[loosers] = np.clip(W[loosers], -1, 1)

        # update Trace
        rate_Trace = np.zeros(I.shape)
        rate_Trace[(I > 0.5)] = 0.3
        rate_Trace[(I < 0.5)] = -0.001

        dTrace = np.zeros(W.shape)
        dTrace[winner] = rate_Trace
        Trace += dTrace
        Trace = np.clip(Trace, 0, 1)

        eps -= eps * rate_drop

print('W', W.shape, W.max(), W.min(), np.median(W))

W (32, 32) 1.0 1.0 1.0
W (32, 32) 1.0 -0.4999999999999995 -0.4999854151338087


In [2370]:
SL_symbols = [x.capitalize() for x in letters][:layer_size]
print(len(SL_symbols))

SLSymbolDict = {}

SL_symbols_d = {}
for idx, val in enumerate(SL_symbols):
    SL_symbols_d.setdefault(val + ':' + str(idx), 0)

upstream_inp_keys = list(upstream_inp.keys())

upstream_cluster_by_symbol: dict[str, int] = {}
for idx, val in enumerate(SL_symbols):
    upstream_cluster_by_symbol.setdefault(val, idx)

for upstr_idx, up_ptr in enumerate(upstream_patterns):
    O = np.dot(W, up_ptr)

    winner = np.argmax(O)
    symbol = SL_symbols[winner]
    print(SL_symbols[winner])

    # print(W[upstream_cluster_by_symbol[symbol]].tolist())

    sls = SL_symbols[winner]
    sls_key = upstream_inp_keys[upstr_idx]
    SLSymbolDict.setdefault(sls_key, sls)
    
    SL_symbols_d[SL_symbols[winner] + ':' + str(winner)] += 1

print(len(SL_symbols_d))
for key, val in SL_symbols_d.items():
    print(key, val)

print(SLSymbolDict)

32
A
O
X
H
J
O
'
I
S
O
E
U
J
.
J
J
J
Y
"
P
N
D
A
Y
J
C
J
J
S
K
'
C
K
E
I
O
E
V
O
.
F
U
O
J
Y
"
P
D
A
G
V
P
T
Z
C
K
J
M
L
.
S
R
S
T
J
R
S
W
B
Z
O
O
 
R
G
,
U
F
U
J
S
L
N
N
V
O
M
F
R
E
X
V
J
M
K
C
S
E
C
.
'
S
S
.
G
,
K
I
"
,
W
X
A
N
S
K
32
A:0 4
B:1 1
C:2 5
D:3 2
E:4 5
F:5 3
G:6 3
H:7 1
I:8 3
J:9 13
K:10 6
L:11 2
M:12 3
N:13 4
O:14 9
P:15 3
Q:16 0
R:17 4
S:18 10
T:19 2
U:20 4
V:21 4
W:22 2
X:23 3
Y:24 3
Z:25 2
 :26 1
.:27 5
,:28 3
":29 3
':30 3
1:31 0
{0: 'A', 2: 'O', 7: 'X', 10: 'H', 12: 'J', 16: 'O', 19: "'", 22: 'I', 26: 'S', 29: 'O', 36: 'E', 39: 'U', 51: 'J', 57: '.', 62: 'J', 67: 'J', 76: 'J', 78: 'Y', 81: '"', 82: 'P', 86: 'N', 90: 'D', 95: 'A', 103: 'Y', 110: 'J', 114: 'C', 118: 'J', 127: 'J', 130: 'S', 132: 'K', 136: "'", 144: 'C', 148: 'K', 152: 'E', 160: 'I', 163: 'O', 170: 'E', 177: 'V', 180: 'O', 183: '.', 186: 'F', 189: 'U', 192: 'O', 199: 'J', 201: 'Y', 204: '"', 205: 'P', 209: 'D', 213: 'A', 215: 'G', 219: 'V', 220: 'P', 224: 'T', 228: 'Z', 230: 'C', 235: 'K', 242: 'J', 2

In [2371]:
sl_input = list(SLSymbolDict.values())
print(sl_input)

['A', 'O', 'X', 'H', 'J', 'O', "'", 'I', 'S', 'O', 'E', 'U', 'J', '.', 'J', 'J', 'J', 'Y', '"', 'P', 'N', 'D', 'A', 'Y', 'J', 'C', 'J', 'J', 'S', 'K', "'", 'C', 'K', 'E', 'I', 'O', 'E', 'V', 'O', '.', 'F', 'U', 'O', 'J', 'Y', '"', 'P', 'D', 'A', 'G', 'V', 'P', 'T', 'Z', 'C', 'K', 'J', 'M', 'L', '.', 'S', 'R', 'S', 'T', 'J', 'R', 'S', 'W', 'B', 'Z', 'O', 'O', ' ', 'R', 'G', ',', 'U', 'F', 'U', 'J', 'S', 'L', 'N', 'N', 'V', 'O', 'M', 'F', 'R', 'E', 'X', 'V', 'J', 'M', 'K', 'C', 'S', 'E', 'C', '.', "'", 'S', 'S', '.', 'G', ',', 'K', 'I', '"', ',', 'W', 'X', 'A', 'N', 'S', 'K']


In [2372]:
# SL train
max_index_L2 = 8

StatsDict_L2, index_by_prev_L2 = predict_train(input_str=sl_input, letters=SL_symbols, max_index=max_index_L2)

In [2373]:
for sdk, sdv in StatsDict_L2.items():
    print(sdk, sdv)
    print('')

A1:O1 {'X': 1}

O1:X1 {'H': 1}

X1:H1 {'J': 1}

H1:J1 {'O': 1}

J1:O2 {"'": 1}

O2:'1 {'I': 1}

'1:I1 {'S': 1}

I1:S1 {'O': 1}

S1:O3 {'E': 1}

O3:E1 {'U': 1}

E1:U1 {'J': 1}

U1:J2 {'.': 1}

J2:.1 {'J': 1}

.1:J3 {'J': 1}

J3:J3 {'J': 1}

J4:J4 {'Y': 1}

J5:Y1 {'"': 1}

Y1:"1 {'P': 1}

"1:P1 {'N': 1}

P1:N1 {'D': 1}

N1:D1 {'A': 1}

D1:A2 {'Y': 1}

A2:Y2 {'J': 1}

Y2:J6 {'C': 1}

J6:C1 {'J': 1}

C1:J7 {'J': 1}

J7:J7 {'S': 1}

J8:S2 {'K': 1}

S2:K1 {"'": 1}

K1:'2 {'C': 1}

'2:C2 {'K': 1}

C2:K2 {'E': 1}

K2:E2 {'I': 1}

E2:I2 {'O': 1}

I2:O4 {'E': 1}

O4:E3 {'V': 1}

E3:V1 {'O': 1}

V1:O5 {'.': 1}

O5:.2 {'F': 1}

.2:F1 {'U': 1}

F1:U2 {'O': 1}

U2:O6 {'J': 1}

O6:J1 {'Y': 1}

J1:Y3 {'"': 1}

Y3:"2 {'P': 1}

"2:P2 {'D': 1}

P2:D2 {'A': 1}

D2:A3 {'G': 1}

A3:G1 {'V': 1}

G1:V2 {'P': 1}

V2:P3 {'T': 1}

P3:T1 {'Z': 1}

T1:Z1 {'C': 1}

Z1:C3 {'K': 1}

C3:K3 {'J': 1}

K3:J2 {'M': 1}

J2:M1 {'L': 1}

M1:L1 {'.': 1}

L1:.3 {'S': 1}

.3:S3 {'R': 1}

S3:R1 {'S': 1}

R1:S4 {'T': 1}

S4:T2 {'

In [2374]:
for sdk, sdv in index_by_prev_L2.items():
    print(sdk, sdv)
    print('')

A1:O {1: 1}

A0:O {1: 1}

O1:X {1: 1}

O0:X {1: 1}

X1:H {1: 1}

X0:H {1: 1}

H1:J {1: 1}

H0:J {1: 1}

J1:O {2: 1}

J0:O {2: 1}

O2:' {1: 1}

O0:' {1: 1}

'1:I {1: 1}

'0:I {1: 1}

I1:S {1: 1}

I0:S {1: 1}

S1:O {3: 1}

S0:O {3: 1}

O3:E {1: 1}

O0:E {1: 1, 3: 1}

E1:U {1: 1}

E0:U {1: 1}

U1:J {2: 1}

U0:J {2: 1, 4: 1}

J2:. {1: 1}

J0:. {1: 1}

.1:J {3: 1}

.0:J {3: 1}

J3:J {3: 1}

J0:J {3: 1, 4: 1, 7: 1}

J4:J {4: 1}

J5:Y {1: 1}

J0:Y {1: 1, 3: 1}

Y1:" {1: 1}

Y0:" {1: 1, 2: 1}

"1:P {1: 1}

"0:P {1: 1, 2: 1}

P1:N {1: 1}

P0:N {1: 1}

N1:D {1: 1}

N0:D {1: 1}

D1:A {2: 1}

D0:A {2: 1, 3: 1}

A2:Y {2: 1}

A0:Y {2: 1}

Y2:J {6: 1}

Y0:J {6: 1}

J6:C {1: 1}

J0:C {1: 1}

C1:J {7: 1}

C0:J {7: 1}

J7:J {7: 1}

J8:S {2: 1}

J0:S {2: 1, 6: 1}

S2:K {1: 1}

S0:K {1: 1}

K1:' {2: 1}

K0:' {2: 1}

'2:C {2: 1}

'0:C {2: 1}

C2:K {2: 1}

C0:K {2: 1, 3: 1}

K2:E {2: 1}

K0:E {2: 1}

E2:I {2: 1}

E0:I {2: 1}

I2:O {4: 1}

I0:O {4: 1}

O4:E {3: 1}

E3:V {1: 1}

E0:V {1: 1}

V1:O {5: 1}

V0:O

In [2375]:
# predict
TF_L2 = []

TF_L2, err_L2, out_L2 = predict(input_str=sl_input, 
                                TF=TF_L2, 
                                StatsDict=StatsDict_L2, 
                                index_by_prev=index_by_prev_L2, 
                                max_index=max_index_L2)

for o in out_L2:
    print(o)

print(err_L2, err_L2 / (len(sl_input) / 100), len(sl_input), len(out_L2) )

X X {'X': 1} [1] {1: 1} [1, 2, 3, 4, 5, 6, 7, 8] False
H H {'H': 1} [1] {1: 1} [1] False
J J {'J': 1} [1] {1: 1} [1] False
O O {'O': 1} [1] {1: 1} [1] False
' ' {"'": 1} [2] {2: 1} [1] False
I I {'I': 1} [1] {1: 1} [2] False
S S {'S': 1} [1] {1: 1} [1] False
O O {'O': 1} [1] {1: 1} [1] False
E E {'E': 1} [3] {3: 1} [1] False
U U {'U': 1} [1] {1: 1} [3] False
J J {'J': 1} [1] {1: 1} [1] False
. . {'.': 1} [2] {2: 1} [1] False
J J {'J': 1} [1] {1: 1} [2] False
J J {'J': 1} [3] {3: 1} [1] False
J J {'J': 1} [3] {4: 1} [3] True
Y Y {'Y': 1} [4] {5: 1} [4] True
" " {'"': 1} [1] {1: 1} [5] False
P P {'P': 1} [1] {1: 1} [1] False
N N {'N': 1} [1] {1: 1} [1] False
D D {'D': 1} [1] {1: 1} [1] False
A A {'A': 1} [1] {1: 1} [1] False
Y Y {'Y': 1} [2] {2: 1} [1] False
J J {'J': 1} [2] {2: 1} [2] False
C C {'C': 1} [6] {6: 1} [2] False
J J {'J': 1} [1] {1: 1} [6] False
J J {'J': 1} [7] {7: 1} [1] False
S S {'S': 1} [7] {8: 1} [7] True
K K {'K': 1} [2] {2: 1} [8] False
' ' {"'": 1} [1] {1: 1} [2] Fa

In [2376]:
def ds_predict_train_without_index(input_str: str | list,
                                   #    sl_input: list,
                                   SLSymbolDict: dict,
                                   #    StatsDict_L1: dict,
                                   #    index_by_prev_L1: dict,
                                   #    max_index_L1: int,
                                   ):
    ds_symbol = '0'
    DsSymbolsStats = {}
    DsSymbolsStatsProb = {}

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        peak_symbol = SLSymbolDict.get(inp_idx, 0)  # inp_idx - 1
        if peak_symbol != 0:
            ds_symbol = peak_symbol

        DsSymbolsStats.setdefault(ds_symbol, {})
        DsSymbolsStats[ds_symbol].setdefault(inp, 0)

        DsSymbolsStats[ds_symbol][inp] += 1

    for ds_inp, inps in DsSymbolsStats.items():
        norm = sum(inps.values())

        for symbol, symbol_stats in inps.items():
            DsSymbolsStatsProb[ds_inp + ':' + symbol] = symbol_stats / norm

    return DsSymbolsStats, DsSymbolsStatsProb

In [2377]:
DsSymbolsStats, DsSymbolsStatsProb = ds_predict_train_without_index(input_str, SLSymbolDict)
print(DsSymbolsStatsProb)

{'0:h': 1.0, 'O:e': 0.17647058823529413, 'O: ': 0.17647058823529413, 'O:s': 0.058823529411764705, 'O:k': 0.0196078431372549, 'O:y': 0.0196078431372549, 'O:p': 0.0196078431372549, 'O:c': 0.0784313725490196, 'O:o': 0.0784313725490196, 'O:l': 0.0196078431372549, 'O:r': 0.058823529411764705, 'O:w': 0.0392156862745098, 'O:d': 0.0392156862745098, 'O:h': 0.0392156862745098, 'O:a': 0.058823529411764705, 'O:t': 0.0392156862745098, 'O:.': 0.0196078431372549, 'O:u': 0.0196078431372549, 'O:b': 0.0196078431372549, 'O:f': 0.0196078431372549, 'X: ': 0.3333333333333333, 'X:a': 0.2222222222222222, 'X:b': 0.1111111111111111, 'X:n': 0.1111111111111111, 'X:d': 0.1111111111111111, 'X:m': 0.1111111111111111, 'H:o': 0.5, 'H:v': 0.5, 'J:e': 0.16666666666666666, 'J: ': 0.16666666666666666, 'J:t': 0.06060606060606061, 'J:h': 0.030303030303030304, 'J:u': 0.030303030303030304, 'J:n': 0.06060606060606061, 'J:d': 0.07575757575757576, 'J:a': 0.12121212121212122, 'J:c': 0.015151515151515152, 'J:l': 0.0151515151515151

In [2378]:
# from itertools import cycle

def ds_predict(input_str, 
               StatsDict: dict, 
               index_by_prev: dict,
               upstream_cluster_by_symbol: dict,
               letters_dict: dict,
               out_L2: list,
               SLSymbolDict: dict,
               W,
               DsSymbolsStats: dict,
               max_index):
    prev_index_l = [x for x in range(1, max_index+1)]

    err = 0
    TF = []
    output = []

    ds_out_idx = 0
    ds_inp = '0'
    # ds_pattern = np.ones(W.shape[1])

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        # downstream
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
                # ds_pattern = np.ones(W.shape[1])
            else:
                ds_inp = out_L2[ds_out_idx]
                # ds_pattern = W[upstream_cluster_by_symbol.get(ds_inp)]
                # ds_pattern = np.copy(W[upstream_cluster_by_symbol.get(ds_inp)])
                # ds_pattern[np.where(ds_pattern < 0)] = 0

                ds_out_idx += 1
        print(stats_d)

        # prob stats
        # norm = sum(stats_d.values())
        # for st_key, st in stats_d.items():
        #     stats_d[st_key] = st / norm

        for st_key, st in stats_d.items():
            # letter_idx = letters_dict.get(st_key)
            
            # ds_stats = ds_pattern[letter_idx]
            ds_key = ds_inp + ':' + st_key
            ds_stats = DsSymbolsStats.get(ds_key, 0)
            print('st_ds:', st_key, st, ds_key, ds_stats)
            if ds_stats <= 0:
                stats_d[st_key] = 0

                continue
            
            stats_d[st_key] *= ds_stats
            print('upd:', ds_stats)
        
        print(stats_d)
        print(input_str[inp_idx + 1], max(stats_d, key=stats_d.get))
        print()

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [2379]:
DsSymbolsStats, DsSymbolsStatsProb = ds_predict_train_without_index(input_str, SLSymbolDict)

dsp_TF, dsp_err, dsp_output = ds_predict(input_str=input_str,
                                         StatsDict=StatsDict,
                                         index_by_prev=index_by_prev_L1,
                                         upstream_cluster_by_symbol=upstream_cluster_by_symbol,
                                         letters_dict=letters_dict,
                                         out_L2=out_L2,
                                         SLSymbolDict=SLSymbolDict,
                                         W=W,
                                         DsSymbolsStats=DsSymbolsStatsProb,
                                         max_index=max_index_L1)

print(dsp_err, dsp_err / (len(input_str) / 100))


{'e': 6, 'i': 1}
st_ds: e 6 0:e 0
st_ds: i 1 0:i 0
{'e': 0, 'i': 0}
e e

{' ': 4, 'r': 1, 'a': 2, 't': 1}
st_ds:   4 O:  0.17647058823529413
upd: 0.17647058823529413
st_ds: r 1 O:r 0.058823529411764705
upd: 0.058823529411764705
st_ds: a 2 O:a 0.058823529411764705
upd: 0.058823529411764705
st_ds: t 1 O:t 0.0392156862745098
upd: 0.0392156862745098
{' ': 0.7058823529411765, 'r': 0.058823529411764705, 'a': 0.11764705882352941, 't': 0.0392156862745098}
   

{'s': 1, 'd': 2, 'p': 1, 'f': 1, 't': 1, 'c': 1}
st_ds: s 1 O:s 0.058823529411764705
upd: 0.058823529411764705
st_ds: d 2 O:d 0.0392156862745098
upd: 0.0392156862745098
st_ds: p 1 O:p 0.0196078431372549
upd: 0.0196078431372549
st_ds: f 1 O:f 0.0196078431372549
upd: 0.0196078431372549
st_ds: t 1 O:t 0.0392156862745098
upd: 0.0392156862745098
st_ds: c 1 O:c 0.0784313725490196
upd: 0.0784313725490196
{'s': 0.058823529411764705, 'd': 0.0784313725490196, 'p': 0.0196078431372549, 'f': 0.0196078431372549, 't': 0.0392156862745098, 'c': 0.0784313

In [2380]:
ds_predict_train_without_index(input_str, SLSymbolDict)

({'0': {'h': 1},
  'O': {'e': 9,
   ' ': 9,
   's': 3,
   'k': 1,
   'y': 1,
   'p': 1,
   'c': 4,
   'o': 4,
   'l': 1,
   'r': 3,
   'w': 2,
   'd': 2,
   'h': 2,
   'a': 3,
   't': 2,
   '.': 1,
   'u': 1,
   'b': 1,
   'f': 1},
  'X': {' ': 3, 'a': 2, 'b': 1, 'n': 1, 'd': 1, 'm': 1},
  'H': {'o': 1, 'v': 1},
  'J': {'e': 11,
   ' ': 11,
   't': 4,
   'h': 2,
   'u': 2,
   'n': 4,
   'd': 5,
   'a': 8,
   'c': 1,
   'l': 1,
   '.': 1,
   '"': 2,
   'r': 4,
   'o': 1,
   's': 3,
   'y': 1,
   ',': 1,
   'g': 1,
   'x': 1,
   'p': 1,
   'i': 1},
  "'": {'o': 2,
   'r': 3,
   't': 2,
   'u': 1,
   'l': 1,
   'd': 2,
   'e': 2,
   ' ': 1,
   'a': 1,
   'z': 1},
  'I': {' ': 2,
   'w': 1,
   'a': 1,
   's': 1,
   't': 2,
   'h': 1,
   'e': 1,
   'i': 1,
   'c': 1},
  'S': {' ': 10,
   't': 2,
   'h': 3,
   'a': 2,
   'v': 1,
   'j': 1,
   'y': 1,
   'o': 1,
   'w': 2,
   's': 1,
   'e': 1,
   'n': 2,
   'd': 1,
   'i': 1,
   'g': 1},
  'E': {' ': 3,
   'o': 4,
   'f': 1,
   't': 1,
   'h

In [2381]:
def downstream_predict_train(input_str: str | list,
                             sl_input: list,
                             SLSymbolDict: dict,
                            #  ds_out: list,
                             StatsDict_L1: dict,
                            #  StatsDict_L2,
                            #  TF_L1,
                             index_by_prev_L1: dict,
                            #  index_by_prev_L2,
                             max_index_L1: int,
                            #  max_index_L2,
                             ):
    next_index: dict[str, int] = {}

    for letter in letters:
        next_index[letter] = 1

    ds_inp = '0'
    sl_inp_idx = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if sl_inp_idx == len(sl_input):
                ds_inp = '0'
            else:
                ds_inp = sl_input[sl_inp_idx]
                sl_inp_idx += 1
        # print(f'{ds_inp=}')
        stats_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict_L1.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict_L1[stats_key].setdefault(next_inp, 0)

        StatsDict_L1[stats_key][next_inp] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(0) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index_L1:
            next_index[prev_inp] = 1
        
    return StatsDict_L1, index_by_prev_L1


In [2382]:
StatsDict_L1_ds = {}
index_by_prev_L1_ds = {}

StatsDict_L1_ds, index_by_prev_L1_ds = downstream_predict_train(input_str=input_str,
                                                                sl_input=sl_input,
                                                                SLSymbolDict=SLSymbolDict,
                                                                StatsDict_L1=StatsDict_L1_ds,
                                                                index_by_prev_L1=index_by_prev_L1_ds,
                                                                max_index_L1=max_index_L1, )

for key, val in StatsDict_L1_ds.items():
    print(key, val)

0:t1:h1 {'e': 1}
O:h1:e1 {' ': 1}
O:e1: 1 {'s': 1}
O: 1:s1 {'k': 1}
O:s1:k1 {'y': 1}
O:k1:y1 {' ': 1}
X:y1: 2 {'a': 1}
X: 2:a1 {'b': 1}
X:a1:b1 {'o': 1}
H:b1:o1 {'v': 1}
H:o1:v1 {'e': 1}
J:v1:e2 {' ': 1}
J:e2: 3 {'t': 1}
J: 3:t2 {'h': 1}
J:t2:h2 {'e': 1}
O:h2:e3 {' ': 1}
O:e3: 4 {'p': 1, 'f': 1}
O: 4:p1 {'o': 1}
':p1:o2 {'r': 1}
':o2:r1 {'t': 1}
':r1:t3 {' ': 1}
I:t3: 5 {'w': 1}
I: 5:w1 {'a': 1}
I:w1:a2 {'s': 1}
I:a2:s2 {' ': 1}
S:s2: 6 {'t': 1}
S: 6:t4 {'h': 1}
S:t4:h3 {'e': 1}
O:h3:e4 {' ': 1}
O:e4: 7 {'c': 1}
O: 7:c1 {'o': 1}
O:c1:o3 {'l': 1}
O:o3:l1 {'o': 1}
O:l1:o4 {'r': 1}
O:o4:r2 {' ': 1}
E:r2: 8 {'o': 1}
E: 8:o5 {'f': 1}
E:o5:f1 {' ': 1}
U:f1: 1 {'t': 1}
U: 1:t5 {'e': 1}
U:t5:e5 {'l': 1}
U:e5:l2 {'e': 1}
U:l2:e6 {'v': 1}
U:e6:v2 {'i': 1}
U:v2:i1 {'s': 1}
U:i1:s3 {'i': 1}
U:s3:i2 {'o': 1}
U:i2:o6 {'n': 1}
U:o6:n1 {',': 1}
U:n1:,1 {' ': 1}
J:,1: 2 {'t': 1}
J: 2:t6 {'u': 1}
J:t6:u1 {'n': 1}
J:u1:n2 {'e': 1}
J:n2:e7 {'d': 1}
J:e7:d1 {' ': 1}
.:d1: 3 {'t': 1}
.: 3:t7 {'o': 1}
.:t7:o

In [2383]:
from itertools import cycle

print(out_L2)
def downstream_predict(input_str: str | list,
                       SLSymbolDict: dict,
                       StatsDict_L1_ds: dict,
                       index_by_prev_L1_ds: dict,
                       out_L2: list,
                       max_index_L1: int,
                       TF: list,
                       err=0,):
    prev_index_l = [x for x in range(1, max_index_L1+1)]

    ds_inp = '0'
    ds_out_idx = 0
    ds_predict_out = cycle(out_L2)

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]

        # print(inp, SLSymbolDict.get(inp_idx, 0))
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_out_idx += 1
        print(ds_inp, is_peak)
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev_L1_ds.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = ds_inp + ':' + prev_inp + \
                        str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict_L1_ds.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index_L1:
                max_key_d = 1

            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)

        prev_index_l = list(index_d.keys())

        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)

    return TF, err


['O', 'X', 'H', 'J', 'O', "'", 'I', 'S', 'O', 'E', 'U', 'J', '.', 'J', 'J', 'J', 'Y', '"', 'P', 'N', 'D', 'A', 'Y', 'J', 'C', 'J', 'J', 'S', 'K', "'", 'C', 'K', 'E', 'I', 'O', 'E', 'V', 'O', '.', 'F', 'U', 'O', 'J', 'Y', '"', 'P', 'D', 'A', 'G', 'V', 'P', 'T', 'Z', 'C', 'K', 'J', 'M', 'L', '.', 'S', 'R', 'S', 'T', 'J', 'R', 'S', 'W', 'B', 'Z', 'O', 'O', ' ', 'R', 'G', ',', 'U', 'F', 'U', 'J', 'S', 'L', 'N', 'N', 'V', 'O', 'M', 'F', 'R', 'E', 'X', 'V', 'J', 'M', 'K', 'C', 'S', 'E', 'C', '.', "'", 'S', 'S', '.', 'G', ',', 'K', 'I', '"', ',', 'W', 'X', 'A', 'N', 'S', 'K']


In [2384]:
TF_L1_ds = []

TF_L1_ds, err_L1_ds = downstream_predict(input_str=input_str,
                                         SLSymbolDict=SLSymbolDict,
                                         StatsDict_L1_ds=StatsDict_L1_ds,
                                         index_by_prev_L1_ds=index_by_prev_L1_ds,
                                         out_L2=out_L2,
                                         max_index_L1=max_index_L1,
                                         TF=TF_L1_ds, )

print(err_L1_ds, err_L1_ds / (len(input_str) / 100))

0 0
O O
O 0
O 0
O 0
O 0
X X
X 0
X 0
H H
H 0
J J
J 0
J 0
J 0
O O
O 0
O 0
' '
' 0
' 0
I I
I 0
I 0
I 0
S S
S 0
S 0
O O
O 0
O 0
O 0
O 0
O 0
O 0
E E
E 0
E 0
U U
U 0
U 0
U 0
U 0
U 0
U 0
U 0
U 0
U 0
U 0
U 0
J J
J 0
J 0
J 0
J 0
J 0
. .
. 0
. 0
. 0
. 0
J J
J 0
J 0
J 0
J 0
J J
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J J
J 0
Y Y
Y 0
Y 0
" "
P P
P 0
P 0
P 0
N N
N 0
N 0
N 0
D D
D 0
D 0
D 0
D 0
A A
A 0
A 0
A 0
A 0
A 0
A 0
A 0
Y Y
Y 0
Y 0
Y 0
Y 0
Y 0
Y 0
J J
J 0
J 0
J 0
C C
C 0
C 0
C 0
J J
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J J
J 0
J 0
S S
S 0
K K
K 0
K 0
K 0
' '
' 0
' 0
' 0
' 0
' 0
' 0
' 0
C C
C 0
C 0
C 0
K K
K 0
K 0
K 0
E E
E 0
E 0
E 0
E 0
E 0
E 0
E 0
I I
I 0
I 0
O O
O 0
O 0
O 0
O 0
O 0
O 0
E E
E 0
E 0
E 0
E 0
E 0
E 0
V V
V 0
V 0
O O
O 0
O 0
. .
. 0
. 0
F F
F 0
F 0
U U
U 0
U 0
O O
O 0
O 0
O 0
O 0
O 0
O 0
J J
J 0
Y Y
Y 0
Y 0
" "
P P
P 0
P 0
P 0
D D
D 0
D 0
D 0
A A
A 0
G G
G 0
G 0
G 0
V V
P P
P 0
P 0
P 0
T T
T 0
T 0
T 0
Z Z
Z 0
C C
C 0
C 0
C 0
C 0
K K
K 0
K 0
K 0
K 0
K 0
K 0
J J
J 0
J 0
J 0
J 0
J 0
M M
M 0
M 0
